In [10]:
import git
from pathlib import Path
import os

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

DATASET = "segmentAnything"
FINAL_DATA_NAME = 'approx1e5-segmentAnything-fourier' # + channel
CONSTANT_SAMPLE_SIZE = int(1e5)
RAW_DATA_SUFFIX = "segmentAnything-resizedBlurred-normalized"
SAVE_DF = False

data_dir = os.path.join(ROOT_DIR, 'raw-data', 'segmentAnything','toy-segmentAnything-resizedBlurred-normalized')
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)
data_dir
BATCH_NUM = None

In [17]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from transform import *
os.chdir(os.path.join(ROOT_DIR, "dataset-preparation"))
freq_df = pd.read_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"), index_col= ["dataset", "transform", "group"])


In [18]:
file_list = [os.path.join(data_dir, f"toy-{RAW_DATA_SUFFIX}", filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(os.path.join(data_dir))
file_names[:5]

['sa_779.npz', 'sa_2365.npz', 'sa_6103.npz', 'sa_9556.npz', 'sa_8890.npz']

In [34]:
'''Assuming No batching is required. Not applicable for agriVision'''

# data_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", "full-agriVision-RGB-cleaned")

# for channel in ['red', 'blue', 'green', 'gray', 'infrared']:

#     channel_fr = convert_to_fourier_basis(data_dir, channel, debug = True)
#     pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"full-agriVision-fourier-{channel}-df.pickle"))

#     min_group, max_group = 2, max(channel_fr['band'])
#     group_data_map = dict()
#     group_data_map_size = dict()
#     for group in np.arange(min_group, max_group + 1):
#         data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
#         group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
#         group_data_map_size[group] = data.size
    
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
    

'Assuming No batching is required. Not applicable for agriVision'

In [35]:
'''To split large dataset into many batches, only needs to be run once'''
# k = 10000
# target_dir = os.path.join(ROOT_DIR, 'raw-data', 'agriVision') # Where the batch{i} folders will be created
# directorySplit(folder_dir = data_dir, target_dir = target_dir, name = RAW_DATA_SUFFIX, k = k)
# print(f"{len(file_names)//k} batches created" )

'To split large dataset into many batches, only needs to be run once'

In [19]:
'''Show all subsets of data in raw data folder that have already been created'''
print(''.join([x+"\n" for x in os.listdir(data_dir) if x.__contains__(RAW_DATA_SUFFIX)]))

In [12]:
def npz_opener_pickle(path):
    image = Image.open(path).convert('RGB')
    return np.array(image)

# Fourier

In [6]:
#Values obtained from plots in coco_indoor_Fourier_NEW.ipynb
STARTING_VALUE = 0.013101960805662831
ENDING_VALUE =0.6162627813472334
MULT_FACTOR = 1.18
if BATCH_NUM is None:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "segmentAnything", f"toy-{RAW_DATA_SUFFIX}")
else:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "coco", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")
splits = getSplits(STARTING_VALUE,ENDING_VALUE, MULT_FACTOR)
print(splits)
BATCH_NUM = None

[0.013101960805662831, 0.01546031375068214, 0.018243170225804923, 0.02152694086644981, 0.025401790222410774, 0.02997411246244471, 0.035369452705684756, 0.04173595419270801, 0.049248425947395445, 0.05811314261792662, 0.0685735082891534, 0.08091673978120101, 0.09548175294181718, 0.11266846847134426, 0.1329487927961862, 0.1568795754994997, 0.18511789908940965, 0.21843912092550338, 0.25775816269209395, 0.30415463197667086, 0.3589024657324716, 0.42350490956431647, 0.4997357932858934, 0.5896882360773542, 0.6958321185712779]


In [22]:
TRANSFORM = "fourier"
channel = "red"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()


100%|██████████| 7072/7072 [04:32<00:00, 25.96it/s]


[0.01310196 0.01546031 0.01824317 0.02152694 0.02540179 0.02997411
 0.03536945 0.04173595 0.04924843 0.05811314 0.06857351 0.08091674
 0.09548175 0.11266847 0.13294879 0.15687958 0.1851179  0.21843912
 0.25775816 0.30415463 0.35890247 0.42350491 0.49973579 0.58968824
 0.69583212]


100%|██████████| 25/25 [01:28<00:00,  3.56s/it]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,red,"(0.0, 0.012506102026236032)",22,"[749.98126, -200.92924, 289.4644, 963.796, 965..."
1,2,red,"(0.013101960805662831, 0.015254393898255184)",7,"[-0.0045912387, -3.550448, 8.43366, -1.8932953..."
2,3,red,"(0.015625, 0.01800692276815017)",10,"[-0.73001677, 12.14289, -3.2347984, 0.50778717..."
3,4,red,"(0.018425744398548053, 0.021484375)",13,"[-0.30569217, -4.0366955, -1.4613433, -1.29072..."
4,5,red,"(0.02157297073669387, 0.025390625)",17,"[2.4934223, -1.136901, 0.62700117, 24.591494, ..."


In [1]:
del channel_fr, group_data_map, group_data_map_size

NameError: name 'channel_fr' is not defined

In [7]:
TRANSFORM = "fourier"
channel = "blue"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 7072/7072 [04:26<00:00, 26.58it/s]


[0.01310196 0.01546031 0.01824317 0.02152694 0.02540179 0.02997411
 0.03536945 0.04173595 0.04924843 0.05811314 0.06857351 0.08091674
 0.09548175 0.11266847 0.13294879 0.15687958 0.1851179  0.21843912
 0.25775816 0.30415463 0.35890247 0.42350491 0.49973579 0.58968824
 0.69583212]


100%|██████████| 25/25 [01:26<00:00,  3.46s/it]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,blue,"(0.0, 0.012506102026236032)",22,"[745.7849, 90.64384, 91.94479, 389.46628, 1237..."
1,2,blue,"(0.013101960805662831, 0.015254393898255184)",7,"[1.375971, 29.873735, -0.5215105, -2.6257505, ..."
2,3,blue,"(0.015625, 0.01800692276815017)",10,"[2.1964629, -6.8183846, -5.704267, 0.8304716, ..."
3,4,blue,"(0.018425744398548053, 0.021484375)",13,"[-2.2812614, 5.8058925, -0.63489276, -1.533044..."
4,5,blue,"(0.02157297073669387, 0.025390625)",17,"[-3.180953, -25.046494, -0.035471905, 31.78473..."


In [8]:
del channel_fr, group_data_map, group_data_map_size

In [21]:
TRANSFORM = "fourier"
channel = "green"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
    
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

NameError: name 'batch_dir' is not defined

In [ ]:
del channel_fr, group_data_map, group_data_map_size

In [ ]:
TRANSFORM = "fourier"
channel = "gray"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
    
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 2446/2446 [00:17<00:00, 143.28it/s]


[0.02620392 0.03092063 0.03648634 0.04305388 0.05080358 0.05994822
 0.07073891 0.08347191 0.09849685 0.11622629 0.13714702 0.16183348
 0.19096351 0.22533694 0.26589759 0.31375915 0.3702358  0.43687824
 0.51551633 0.60830926 0.71780493]


100%|██████████| 21/21 [00:02<00:00,  7.15it/s]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,gray,"(0.0, 0.025012204052472065)",22,"[30452.178, 29851.316, 35510.652, 30514.885, 3..."
1,2,gray,"(0.026203921611325662, 0.030508787796510367)",7,"[129.36272, -245.97491, -56.075764, 226.1529, ..."
2,3,gray,"(0.03125, 0.03601384553630034)",10,"[-112.38401, -302.9333, 585.2108, -261.73734, ..."
3,4,gray,"(0.036851488797096106, 0.04296875)",13,"[-133.53929, -72.047966, 70.92154, 85.77455, -..."
4,5,gray,"(0.04314594147338774, 0.05078125)",17,"[166.04921, 426.75668, -2.0364027, 120.562, -6..."


In [ ]:
del channel_fr, group_data_map, group_data_map_size

# Wavelet

In [38]:
FINAL_DATA_NAME = 'approx1e5-segmentAnything-wavelet'
if BATCH_NUM is None:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "segmentAnything", f"full-{RAW_DATA_SUFFIX}")
else:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "segmentAnything", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")

In [39]:
TRANSFORM = "wavelet"
channel = "red"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener_pickle)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
                             
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

10 layers being used


100%|██████████| 276/276 [00:02<00:00, 93.20it/s] 


,channel,layer,frequency,orientation,data
0,red,1,0.099611,L1,"[66981.18, 53158.035, 89715.58, 47087.85, 6780..."
1,red,2,0.110679,D,"[4121.6465, 555.51953, 3105.1602, 2641.3438, 9..."
2,red,2,0.110679,H,"[2916.5488, -2862.7188, 924.26953, 2968.9375, ..."
3,red,2,0.110679,V,"[4608.537, -14710.102, -540.96094, 4271.242, -..."
4,red,3,0.124514,D,"[5286.629, -159.23438, -738.58984, 1508.1133, ..."


In [22]:
del channel_wv, group_data_map, group_data_map_size

NameError: name 'channel_wv' is not defined

In [23]:
TRANSFORM = "wavelet"
channel = "green"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener_pickle)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

NameError: name 'batch_dir' is not defined

In [42]:
del channel_wv, group_data_map, group_data_map_size

In [43]:
TRANSFORM = "wavelet"
channel = "blue"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener_pickle)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
    
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

10 layers being used


100%|██████████| 276/276 [00:02<00:00, 123.05it/s]


,channel,layer,frequency,orientation,data
0,blue,1,0.099611,L1,"[30034.572, 42459.273, 61878.895, 48168.86, 65..."
1,blue,2,0.110679,D,"[6817.1387, -54.958984, 1830.6484, 3407.4668, ..."
2,blue,2,0.110679,H,"[-1329.6699, -3652.42, -154.94922, 5832.1934, ..."
3,blue,2,0.110679,V,"[-123.64258, -10867.658, 754.8672, 10115.623, ..."
4,blue,3,0.124514,D,"[6257.5977, 61.171875, -1313.8789, 2956.004, 4..."


In [44]:
del channel_wv, group_data_map, group_data_map_size

In [45]:
TRANSFORM = "wavelet"
channel = "gray"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener_pickle)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

10 layers being used


100%|██████████| 276/276 [00:02<00:00, 132.77it/s]


,channel,layer,frequency,orientation,data
0,gray,1,0.099611,L1,"[58814.01, 49292.59, 86745.51, 48341.098, 6952..."
1,gray,2,0.110679,D,"[2538.7622, -282.08478, 2133.065, 3333.022, 83..."
2,gray,2,0.110679,H,"[1385.2566, -2804.156, 626.9482, 4419.2036, 10..."
3,gray,2,0.110679,V,"[1154.8969, -14182.402, 314.66733, 6913.5093, ..."
4,gray,3,0.124514,D,"[3210.0942, -12.312569, -897.23016, 1922.8593,..."


In [46]:
del channel_wv, group_data_map, group_data_map_size